# Preprocessing

In [207]:

import nltk
import string
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import scipy
from keras.preprocessing.text import Tokenizer

nltk.download('punkt')
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

train_file = "train.data.txt"
dev_file = "dev.data.txt"
test_file = "test.data.txt"

train_data = pd.read_csv('./%s.csv'%train_file,keep_default_na=False)
dev_data = pd.read_csv('./%s.csv'%dev_file,keep_default_na=False)
test_data = pd.read_csv('./%s.csv'%test_file,keep_default_na=False)

def preprocess(pd_data):
  data = pd_data.copy()
  data['main_tweet'] = data['main_tweet'].fillna('')
  data.replace('', np.nan, inplace=True)
  data.dropna(subset=['main_tweet'], inplace=True)
  data['main_tweet'] = data['main_tweet'].apply(lambda x: x.lower())
  data['main_tweet'] = data['main_tweet'].apply(lambda x: x.replace('\n', ' ')) 
  return data
le = LabelEncoder()
train_data = preprocess(train_data)
train_label = le.fit_transform(train_data['label'])

dev_data = preprocess(dev_data)
dev_label = le.transform(dev_data['label'])


tokenizer = Tokenizer(oov_token="<UNK>")
tokenizer.fit_on_texts(train_data['main_tweet'])

# Using BAG OF WORDS
train = tokenizer.texts_to_matrix(train_data['main_tweet'], mode="count")
dev = tokenizer.texts_to_matrix(dev_data['main_tweet'], mode="count")
test = tokenizer.texts_to_matrix(test_data['main_tweet'], mode="count")




[nltk_data] Downloading package punkt to /home/kan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


(1566, 6399)

In [208]:
import tensorflow as tf
from tensorflow.keras import losses

if tf.test.gpu_device_name(): 
  print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))

else:
  print("Please install GPU version of TF")
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss=losses.BinaryCrossentropy(from_logits=True), optimizer='adam', metrics=['accuracy'])
model.summary()

history = model.fit(train, train_label, epochs=20, validation_data=(dev, dev_label), batch_size=10)

INFO:plaidml:Opening device "llvm_cpu.0"


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                409600    
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 66        
Total params: 411,746
Trainable params: 411,746
Non-trainable params: 0
_________________________________________________________________


In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 5))
plt.subplot(121)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss']);
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(['train loss', 'test loss']);
ax = plt.subplot(122)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(['train accuracy', 'test accuracy']);

In [ ]:
prediction = model.predict(test)
prediction = (prediction > 0.5).astype("int32")
prediction = np.ndarray.flatten(prediction)
pd.DataFrame({"Predicted":  prediction}).to_csv('submission.csv', index_label="Id")